# Introduction 

...

Before you get started, run the code cell below to set everything up.

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPoint, MultiPolygon
from datetime import datetime

import math

import folium
from folium import Choropleth
from folium.plugins import HeatMap, MarkerCluster

# Exercises

### 1) Create a buffer.

62 hospitals

In [ ]:
hospitals = gpd.read_file("geospatial-course-data/nyu_2451_34494/nyu_2451_34494.shp")
hospitals['latitude'] = hospitals.geometry.to_crs(epsg=4326).y
hospitals['longitude'] = hospitals.geometry.to_crs(epsg=4326).x
hospitals.head()

notes on [buffer](https://shapely.readthedocs.io/en/latest/manual.html#object.buffer)

In [ ]:
hosp_buffer = gpd.GeoDataFrame(hospitals, geometry=hospitals.geometry.buffer(1000))
hosp_buffer.head()

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals_gdf.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(folium.Marker([row['latitude'], row['longitude']]))
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)
        
m.add_child(mc)
m 

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start = 11) 

hospitals_gdf_copy = hospitals_gdf.copy()
hospitals_gdf_copy['count'] = 1
HeatMap(data=hospitals_gdf_copy[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=20, max_zoom=13).add_to(m)

m 

In [ ]:
# testing some stuff
polygon1 = Polygon([(0,0), (1.5,0), (1,1), (0,1), (0,0)])
polygon2 = Polygon([(1,1), (2,1), (2,2), (1,2), (1,1)])
polygon3 = Polygon([(1,0), (2,0), (2,1), (1,1), (1,0)])
polygon_list = [polygon1,polygon2, polygon3]

mpolygon = MultiPolygon(polygon_list)

In [ ]:
mpolygon

In [ ]:
mpolygon.buffer(0)

In [ ]:
# this is the best option
gpd.GeoSeries(polygon_list).unary_union

In [ ]:
MultiPolygon(list(buffer_gdf.geometry.values)).buffer(0).contains(hospitals_gdf.geometry[0])

In [ ]:
#[MultiPolygon(list(buffer_gdf.geometry.values)).buffer(0).contains(i) for i in hospitals_gdf.geometry]

In [ ]:
collisions_df = pd.read_csv("geospatial-course-data/NYPD_Motor_Vehicle_Collisions.csv")
collisions_gdf = gpd.GeoDataFrame(collisions_df, geometry=gpd.points_from_xy(collisions_df["LONGITUDE"], collisions_df["LATITUDE"]))
collisions_gdf.head()

note to self. not every collision needs hospitalization

In [ ]:
danger_gdf = collisions_gdf[collisions_gdf[['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']].sum(axis=1)>0]

In [ ]:
len(danger_gdf)

In [ ]:
from time import time

m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals_gdf.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)

start = time()

for idx, row in danger_gdf.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(folium.Marker([row['LATITUDE'], row['LONGITUDE']]))
m.add_child(mc)

end = time()

print(end-start)

In [ ]:
embed_map(m)

In [ ]:
from tqdm import tqdm
mp = MultiPolygon(list(buffer_gdf.geometry.values)).buffer(0)
inside_range = [mp.contains(i) for i in tqdm(danger_gdf.geometry.values)]

In [ ]:
outside_danger_gdf = danger_gdf[[not i for i in inside_range]]
len(outside_danger_gdf)

In [ ]:
from time import time

m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals_gdf.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)

start = time()

for idx, row in outside_danger_gdf.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(folium.Marker([row['LATITUDE'], row['LONGITUDE']]))
m.add_child(mc)

end = time()

print(end-start)

In [ ]:
embed_map(m)

initially made heatmap with count column all ones. bad idea again because all accidents treated equally, whereas we know some are more dangerous than others

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start = 11) 

danger_gdf_copy = danger_gdf.copy()
danger_gdf_copy['count'] = danger_gdf_copy[[
    #'NUMBER OF PERSONS INJURED', 
    'NUMBER OF PERSONS KILLED',
    #'NUMBER OF PEDESTRIANS INJURED', 
    'NUMBER OF PEDESTRIANS KILLED',
    #'NUMBER OF CYCLIST INJURED', 
    'NUMBER OF CYCLIST KILLED',
    #'NUMBER OF MOTORIST INJURED', 
    'NUMBER OF MOTORIST KILLED'
    ]].sum(axis=1)

HeatMap(data=danger_gdf_copy[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), radius=20, max_zoom=1).add_to(m)

for idx, row in hospitals_gdf.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue'
                    ).add_to(m)

embed_map(m) 

what percentage of car accidents w/ at least one injury or fatality were within 1 km of a hospital?
you can add x new hospitals.  how to increase this number to greater than some percentage?

[closest point](https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe)